In [1]:
import pandas as pd
import glob
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.width", 2000)

In [2]:
import numpy as np
np.set_printoptions(precision=4,suppress=True)
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import nltk 

In [3]:
import spacy

In [4]:
from tqdm import tqdm

In [5]:
nlp = spacy.load('de', disable=['parser', 'tagger'])
# FOR FastText vectors
# nlp.vocab.from_disk('./vocab/')

In [6]:
frames = []
for f in glob.glob('../data/*.csv'):
    frames.append(pd.read_csv(f))

In [7]:
raw_df = pd.concat(frames)

In [8]:
raw_df

,SichterGruppe,SichterName,Host,kind,Datetime,Content,Url,Authorid,Tag,entryid,Id,TwistId,TopicId,TopicName,IsInitialEntry,IsPrivate
0,NaN,NaN,www.facebook.com,1,07.04.2017 19:58:19,"Liebe Telekom , vielen Dank das sie unser Anliegen mal wieder nicht gelöst haben . Ich mein - ist in Ordnung , es geht erst seit ca 2 Jahren um einen nicht funktionierenden Anschluss meines Schwiegervaters . Und klar , solange man blöd genug ist und monatlich zahlt ist ja alles gut . Ich weiss leider nicht was die grösste Frechheit ist . Der versprochene Techniker zwischen 13 und 17 Uhr am heutigen Tag ( wär auch schon der x-te ) - welcher aber weder erschienen ist , oder sich abgemeldet hat - oder soeben der Anruf wie zufrieden ich mit dem Service war und wie gut mein Anliegen gelöst wurde ... makaber , finden sie nicht ? Darf ich Ihnen meinen heutigen Arbeitsausfall in Rechnung stellen ? Wären jetzt 200 , - - € auf die 4 Stunden ... Ich bin gespannt wann es mal eine Lösung git und wie die Entschädigung für meinen Schwiegervater und seine Mutter aussieht !",https://www.facebook.com/telekomerleben/posts/1320196341349923#pid=110926408943595&uid=110926408943595_1320196341349923&aid=10212937808460501&vid=2&wid=110926408943595,10786582.0,NaN,991a352b-036d-49f8-96eb-26abf6db27e1,38619209,02eb4808-fbeb-4cd4-a0b2-0000db536177,551631,Alle,True,False
1,NaN,NaN,www.facebook.com,2,08.04.2017 17:37:46,"WEnn es ein Geschäftsanschluss ist und Du / der Schwiegervater den Verlust beweisen kannst , ... Bei einem Privatanschluss wirst Du nichts bekommen .",https://www.facebook.com/telekomerleben/posts/1320196341349923#pid=110926408943595_1320196341349923&uid=1320196341349923_1321024901267067&cpid=&aid=1405233389806347&vid=2&wid=110926408943595,5659126.0,NaN,d227bad0-fcf4-4100-aba6-6406cceb9ed3,38660590,02eb4808-fbeb-4cd4-a0b2-0000db536177,551631,Alle,False,False
2,NaN,NaN,www.facebook.com,2,08.04.2017 17:42:05,Ich rede von meinen 4 Stunden in denen ich auf den Techniker gewartet hab - lässt sich nachweisen ;),https://www.facebook.com/telekomerleben/posts/1320196341349923#pid=110926408943595_1320196341349923&uid=1320196341349923_1321027751266782&cpid=&aid=10212937808460501&vid=2&wid=110926408943595,10786582.0,NaN,053ca782-bb9d-4c20-bad5-aa92099fbc8d,38660749,02eb4808-fbeb-4cd4-a0b2-0000db536177,551631,Alle,False,False
3,NaN,NaN,www.facebook.com,3,10.04.2017 10:24:35,Wir ich sehe hast du deine Rufnummer bereits an Telekom-hilft gesandt . Die Kollegen melden sich schnellstmöglich bei dir .,https://www.facebook.com/telekomerleben/posts/1320196341349923#pid=110926408943595_1320196341349923&uid=1320196341349923_1322498414453049&cpid=&aid=110926408943595&vid=2&wid=110926408943595,254008.0,NaN,082b0370-75ec-41b2-ba22-c4a2a1e2b005,38708945,02eb4808-fbeb-4cd4-a0b2-0000db536177,551631,Alle,False,False
4,NaN,NaN,www.facebook.com,4,10.04.2017 10:26:44,"Du musst den eventuellen "" Schaden "" nachweisen . Das herumsitzen ist kein Schaden",https://www.facebook.com/telekomerleben/posts/1320196341349923#pid=110926408943595_1320196341349923&uid=1320196341349923_1322499154452975&cpid=1320196341349923_1321027751266782&aid=1405233389806347&vid=2&wid=110926408943595,5659126.0,NaN,ffca4732-2832-4140-a7c1-ad18a888db6a,38709009,02eb4808-fbeb-4cd4-a0b2-0000db536177,551631,Alle,False,False
5,Sichter COM,MMS COM,twitter.com,1,22.04.2017 20:52:54,Dies ist das Ergebnis meines Telekom Speedtests ... https://t.co/wodN8YJDYE via,https://twitter.com/KolyatheKoddl/status/855856956514414593#aid=2980145476,10935647.0,abgelegt durch Sichter,17c56e3d-c6a1-4adc-a49d-fb38b217be8d,39287587,837843fd-c1e2-40b9-9108-0003ebe32a63,1,Client 388,True,False
6,Sichter FB/TW,Mobilfunk,twitter.com,1,12.03.2017 12:34:38,"@Telekom_hilft , brauche nochmal hochwertigen Support i . S . Wunschruf-Nr .. 3x Telefonat , 3x versch . Antworten - wo kann ich mich melden ?",https://twitter.com/stadtwaechter/status/840888761865105408#aid=741519973319856128,5927804.0,NaN,4451aef9-e612-4662-a0b3-74b274852ee9,37623335,110f5d4e-4ef4-44

In [25]:
pd.unique(raw_df.SichterName)

array([nan, 'MMS COM', 'Mobilfunk', 'Festnetz', 'MMS',
       'Mobilfunk (Mark./T-Shop)', 'Mobilfunk COM',
       'Festnetz (Mark./T-Shop)', 'Festnetz COM', 'GK',
       'MMS (Mark./T-Shop)', 'Hybrid FB/TW', 'Wechsler', 'Hotspot',
       'Smart Home FB/TW', 'Meinungsbildner', 'A-Team', 'Hosting FB/TW',
       'Hybrid & LTE', 'Abuse/Missbrauch', 'Mobile',
       'Anfragen in Arabisch', 'Entertain', 'IP & FTTH & PSTN',
       'Neue Themen', 'Dienste & Abos', 'Hosting', 'Endgeräte'],
      dtype=object)

In [9]:
df = pd.concat([raw_df[raw_df.Tag.isnull()], raw_df[raw_df.Tag=='Prio-Fall']])
df = df[df.kind == 1]

In [10]:
import re
GRUBER_URLINTEXT_PAT = re.compile(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?\xab\xbb\u201c\u201d\u2018\u2019]))')
NUMBER_CLEAN = re.compile(r'\d+(:?[.,]?\d+)*')
TIME_CLEAN = re.compile(r'\d+:\d+(?:\s?[Uu]hr)?')

def preprocessing(text):
    try:
        # clean-up url
        text = GRUBER_URLINTEXT_PAT.sub('url', text)
        # remove time
        text = TIME_CLEAN.sub('zeit', text)
        # clean-up numbers
        text = NUMBER_CLEAN.sub('nummer', text)
        return text
    except:
        return ''

preprocessing('Yeah , der WG gefällt deine Musik . Hol dir auch superschnelles Internet ! https://t.co/0dPArCUv0b https://t.co/HHRZaAQnte http://pbs.twimg.com/amplify_video_thumb/0123456789001234567890/img/QEQntlwB_ifYH3Gv.jpg')

with open('./stopwords-de.txt') as f:
    stopwords = f.read().split('\n')
    
def filter_toks(token):
    if token.tag_ in ['$,','$.','$(']:
        return False
    if token.pos_ in ['ADP']:
        return False
    if token.lemma_.lower() in stopwords:
        return False
    return True

In [11]:
documents = [] # Content
docs = [] # Content through spacy
labels = [] # SichterName
tags = [] # Tag

for (idx, row) in tqdm(df.iterrows()):
    labels.append(row.SichterName)
    documents.append(preprocessing(row.Content))
#     docs.append([token for token in nlp(documents[-1])])
    tags.append(str(row.Tag))

109032it [00:18, 5795.33it/s]


In [ ]:
labels[0], documents[0], docs[0], tags[0]

# RASA TRAINING DATA

In [308]:
import simplejson
fp = {
    "rasa_nlu_data": {
        "common_examples": [],
        "regex_features" : [],
        "entity_synonyms": []
    }
}
fb = {
    "rasa_nlu_data": {
        "common_examples": [],
        "regex_features" : [],
        "entity_synonyms": []
    }
}

for i in range(len(documents)):
    fp["rasa_nlu_data"]['common_examples'].append(dict(text=documents[i], intent=tags[i], entities=[]))

with open('train_prio.json', 'wt') as file_p:
    simplejson.dump(fp, file_p)
    
# with open('train_bucket.json', 'w') as file_b:
#     json.dump(fb, file_b)

In [276]:
{label for label in labels if type(label) is str}
len([label for label in labels if type(label) is str])

190

In [283]:
print('prio distrib.: {}'.format(sum(tags)/len(tags)))

prio distrib.: 0.10005817335660268


In [ ]:
# TODO: try different sets of stopwords (e.g. with or without kein, nicht)

In [261]:
with open('./train_bucket.json') as f:
    test=json.load(f)

In [267]:
test['common_examples']

{'intent': nan, 'text': 'Ich warte noch auf eine korrekte Rechnung !'}